# 🧮 Non-Dimensional Incompressible Navier-Stokes Equations (2D, Scalar Form)

#### 🔹 Continuity Equation:
$$
\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} = 0
$$
#### 🔹 x-Momentum Equation:
$$
\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y} = -\frac{\partial p}{\partial x} + Pr \left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right)
$$
#### 🔹 y-Momentum Equation:
$$
\frac{\partial v}{\partial t} + u \frac{\partial v}{\partial x} + v \frac{\partial v}{\partial y} = -\frac{\partial p}{\partial y} + Pr \left( \frac{\partial^2 v}{\partial x^2} + \frac{\partial^2 v}{\partial y^2} \right) + Pr*Ra*T
$$
#### 🔹 Non-Dimensional Heat Equation (with Advection)
$$
\frac{\partial T}{\partial t} + u \frac{\partial T}{\partial x} + v \frac{\partial T}{\partial y} =  \frac{\partial^2 T}{\partial x^2} + \frac{\partial^2 T}{\partial y^2} 
$$

# 🔁 Vorticity–Streamfunction Form (2D, Non-Dimensional)

#### 1. Vorticity Definition:
$$
\omega = -\frac{\partial v}{\partial x} + \frac{\partial u}{\partial y}
$$

$$
\omega = \nabla^2 \psi = \left( \frac{\partial^2 \psi}{\partial x^2} + \frac{\partial^2 \psi}{\partial y^2} \right)
$$

#### 2. Vorticity Transport Equation (from momentum equations):
$$
\frac{\partial \omega}{\partial t} + u \frac{\partial \omega}{\partial x} + v \frac{\partial \omega}{\partial y} = Pr \left( \frac{\partial^2 \omega}{\partial x^2} + \frac{\partial^2 \omega}{\partial y^2} \right) + Pr \cdot Ra \cdot \frac{\partial T}{\partial x}
$$

#### 3. Streamfunction–Vorticity Relation (Poisson Equation):
$$
\nabla^2 \psi = \omega
$$

#### 4. Scalar Heat Equation:
$$
\frac{\partial T}{\partial t} + u \frac{\partial T}{\partial x} + v \frac{\partial T}{\partial y} = \frac{\partial^2 T}{\partial x^2} + \frac{\partial^2 T}{\partial y^2}
$$

In [ ]:
# Importing dependencies
import numpy as np
# Defining parameters
N = 61 # 121 # Number of points along x axis
dx = 1.0 / (N - 1) # Grid spacing in x direction
M = 61 # 121 # Number of points along y axis
dy = 1.0 / (M - 1) # Grid spacing in y direction
Ra = 3.5e3 # 2.5e4 # Rayleigh number
Pr = 0.7 # Prandtl number
CFL_crit = 2 * np.sqrt(2) # Critical CFL number for stability
margin_h = 0.1 # Margin for the time step
h = margin_h * CFL_crit / (1/dx + 1/dy) # Time step
beta = 1.5 # relaxation factor for iterative methods to solve algebraic equations
'''
For the modified RK4 method, the CFL number has to be smaller than 2*sqrt(2)
The CFL number is defined as:
    CFL = u*dt/dx + v*dt/dy
The maximum velocity is taken to be 1.0
Hence the time step is given by:
    dt = CFL*dx/u = 

'''

# Initialisation at t=0 with boundary conditions
u = np.zeros((N, M)) # x-velocity
v = np.zeros((N, M)) # y-velocity
T = np.zeros((N, M)) # Temperature
vor = np.zeros((N, M)) # Vorticity
p = np.zeros((N, M)) # Stream function
for i in range(N):
    T[i, 0] = 0.5 * np.cos(np.pi * i / (N-1))+1 # Bottom boundary condition of T = 0.5cos(pi*x)+1


# Solution of Vorticity and Convection equations
## ✨ Creating Vorticity Residual Function (or spatial operator $S(\omega_{i,j})$)
$$
R^\omega =  Pr \left( \frac{\partial^2 \omega}{\partial x^2} + \frac{\partial^2 \omega}{\partial y^2} \right) + Pr \cdot Ra \cdot \frac{\partial T}{\partial x} - u \frac{\partial \omega}{\partial x} - v \frac{\partial \omega}{\partial y}
$$


$$
R^\omega_{i,j} = Pr\left(\frac{\omega_{i-1,j}-2\omega_{i,j}+\omega_{i+1,j}}{\Delta x^2}+\frac{\omega_{i,j-1}-2\omega_{i,j}+\omega_{i,j+1}}{\Delta y^2}\right) + Pr \cdot Ra \cdot \frac{T_{i+1,j} - T_{i-1,j}}{2\Delta x} - u_{i,j} \cdot \frac{\omega_{i+1,j} - \omega_{i-1,j}}{2\Delta x} - v_{i,j} \cdot \frac{\omega_{i,j+1} - \omega_{i,j-1}}{2\Delta y}
$$

## ✨ Creating Temperature Residual Function (or spatial operator $S(T_{i,j})$)
$$
R^T = \frac{\partial^2 T}{\partial x^2} + \frac{\partial^2 T}{\partial y^2} - u \frac{\partial T}{\partial x} - v \frac{\partial T}{\partial y}
$$

$$
R^T_{i,j} = \frac{T_{i-1,j} - 2T_{i,j} + T_{i+1,j}}{\Delta x^2} + \frac{T_{i,j-1} - 2T_{i,j} + T_{i,j+1}}{\Delta y^2} - u_{i,j} \cdot \frac{T_{i+1,j} - T_{i-1,j}}{2\Delta x} - v_{i,j} \cdot \frac{T_{i,j+1} - T_{i,j-1}}{2\Delta y}
$$


In [ ]:
def resvor(vor):
    rvor = np.zeros_like(vor)
    for i in range(1, N - 1):
        for j in range(1, M - 1):
            dvorx2 = (vor[i+1, j] - 2*vor[i, j] + vor[i-1, j]) / dx**2
            dvory2 = (vor[i, j+1] - 2*vor[i, j] + vor[i, j-1]) / dy**2
            dvorx1 = u[i, j] * (vor[i+1, j] - vor[i-1, j]) / (2*dx)
            dvory1 = v[i, j] * (vor[i, j+1] - vor[i, j-1]) / (2*dy)
            dTx = (T[i+1, j] - T[i-1, j]) / (2*dx)

            rvor[i, j] = (dvorx2 + dvory2) * Pr + Pr * Ra * dTx - dvorx1 - dvory1
            
    return rvor

def restemp(T):
    rtemp = np.zeros_like(T)
    for i in range(1, N - 1):
        for j in range(1, M - 1):
            dTx2 = (T[i+1, j] - 2*T[i, j] + T[i-1, j]) / dx**2
            dTy2 = (T[i, j+1] - 2*T[i, j] + T[i, j-1]) / dy**2
            dTx1 = u[i, j] * (T[i+1, j] - T[i-1, j]) / (2*dx)
            dTy1 = v[i, j] * (T[i, j+1] - T[i, j-1]) / (2*dy)

            rtemp[i, j] = dTx2 + dTy2 - dTx1 - dTy1
            
    return rtemp

# ✨ Explicit Euler or Runge-Kutta 4 for Vorticity, $\omega_{i,j}$ (and similarly for Temperature $T_{i,j}$)

## 🔹 Explicit Euler Scheme:
$$
\omega^{n+1}_{i,j} = \omega^n_{i,j} + \Delta t \cdot R^{n}_{i,j}
$$

## 🔹 Explicit Runge-Kutta 4 (RK4) Scheme (Modified proposed by James):
1. Compute intermediate stages:
    $$  
    k_1 = 0.25 \cdot \Delta t \cdot R(\omega^n_{i,j})
    $$
    $$
    k_2 = \frac{\Delta t}{3} \cdot R\left(\omega^n_{i,j} + k_1\right)
    $$
    $$
    k_3 = 0.5 \cdot \Delta t \cdot R\left(\omega^n_{i,j} + k_2\right)
    $$
2. Update:
    $$
    \omega^{n+1} = \Delta t \cdot R\left(\omega^n_{i,j} + k_3\right)
    $$

## 🔹 Explicit Runge-Kutta 4 (RK4) Scheme:
1. Compute intermediate stages:
    $$
    k_1 = \Delta t \cdot R(\omega^n_{i,j})
    $$
    $$
    k_2 = \Delta t \cdot R\left(\omega^n_{i,j} + \frac{1}{2} \cdot k_1\right)
    $$
    $$
    k_3 = \Delta t \cdot R\left(\omega^n_{i,j} + \frac{1}{2} \cdot k_2\right)
    $$
    $$
    k_4 = \Delta t \cdot R\left(\omega^n_{i,j} + 1 \cdot k_3\right)
    $$

2. Update:
    $$
    \omega^{n+1}_{i,j} = \omega^n_{i,j} + \frac{1}{6} \cdot (k_1 + 2k_2 + 2k_3 + k_4)
    $$

In [23]:
def step(var, method="rk4_modified", ResFunction=resvor):
    if method == "euler":
        rvar = ResFunction(var)
        var[1:N-1, 1:M-1] += h * rvar[1:N-1, 1:M-1]
    
    elif method == "rk4_modified":
        vori = np.copy(var)

        # 1st stage
        rvar = ResFunction(var)
        vori[1:N-1, 1:M-1] = var[1:N-1, 1:M-1] + 0.25 * h * rvar[1:N-1, 1:M-1]
        # 2nd stage
        rvar = ResFunction(vori)
        vori[1:N-1, 1:M-1] = var[1:N-1, 1:M-1] + (h / 3.0) * rvar[1:N-1, 1:M-1]
        # 3rd stage
        rvar = ResFunction(vori)
        vori[1:N-1, 1:M-1] = var[1:N-1, 1:M-1] + 0.5 * h * rvar[1:N-1, 1:M-1]
        # 4th stage
        rvar = ResFunction(vori)
        var[1:N-1, 1:M-1] += h * rvar[1:N-1, 1:M-1]
    
    elif method == "rk4":

        # 1st stage
        K1 = h * ResFunction(var)
        # 2nd stage
        K2 = h * ResFunction(var + 0.5 * K1)
        # 3rd stage
        K3 = h * ResFunction(var + 0.5 * K2)
        # 4th stage
        K4 = h * ResFunction(var + K3)

        var[1:N-1, 1:M-1] += (K1 + 2*K2 + 2*K3 + K4) / 6.0
    else:
        raise ValueError("Unknown method: {}".format(method))
    return var

# Solution of Stream function $\psi_{i,j}^{n+1}$ 

$$
\nabla^2 \psi = \frac{\partial^2 \psi}{\partial x^2} + \frac{\partial^2 \psi}{\partial y^2} = \omega
$$

$$
\frac{\psi_{i-1,j}^{n+1} - 2\psi_{i,j}^{n+1} + \psi_{i+1,j}^{n+1}}{\Delta x^2} + \frac{\psi_{i,j-1}^{n+1} - 2\psi_{i,j}^{n+1} + \psi_{i,j+1}^{n+1}}{\Delta y^2} = \omega_{i,j}^{n+1}
$$


## 🔹 Jacobi relaxation solver:

1. Calculate residual of the Poisson equation
    $$
    S_{i,j}^n = \omega_{i,j}^{n+1} - \left(\frac{\psi_{i-1,j}^{n} - 2\psi_{i,j}^{n} + \psi_{i+1,j}^{n}}{\Delta x^2} + \frac{\psi_{i,j-1}^{n} - 2\psi_{i,j}^{n} + \psi_{i,j+1}^{n}}{\Delta y^2}\right )
    $$
2. Calculate coefficients
    $$
    b_W = \frac{1}{\Delta x^2}\\
    b_E = b_W\\
    b_S = \frac{1}{\Delta y^2}\\
    b_N = b_S\\
    b_P = 2 (b_W + b_S)
    $$
3. Calculate $\psi_{i,j}^{n+1}$:
    $$
    b_P \Delta \psi_{i,j}^{n+1} = S_{i,j}^n\\
    \psi_{i,j}^{n+1} = \psi_{i,j}^{n} + \beta \cdot \Delta \psi_{i,j}^{n+1}
    $$

In [ ]:
# Calculation of residual of the Poisson equation
def resp(vor, p, conditions=2):
    rp = np.zeros_like(p)
    if conditions == 1:
        range_x = range(1, N-1)
        range_y = range(1, M-1)
    elif conditions == 2:
        range_x = range(2, N-2)
        range_y = range(2, M-2)
    else:
        raise ValueError("Invalid number of conditions. Use 1 or 2.")

    for i in range_x:
        for j in range_y:
            rp[i, j] = vor[i, j] - (
                (p[i+1, j] - 2*p[i, j] + p[i-1, j]) / dx**2 +
                (p[i, j+1] - 2*p[i, j] + p[i, j-1]) / dy**2
            )
    
    return rp

# Calculate \psi_{i,j}^{n+1} (TO BE DONE AFTER UPDATING VORTICITY!)
def solp(vor, p, beta, conditions=2, method = 'point Jacobi relaxation'):
    # Calculate residual of the Poisson equation
    rp = resp(vor, p, conditions)
    
    # Coefficients for iterative methods
    b_W = 1 / dx**2
    b_E = b_W
    b_S = 1 / dy**2
    b_N = b_S
    b_P = 2 * (b_W + b_S)

    # Defining grid to update based on boundary conditions of \psi
    if conditions == 1:
        range_x = range(1, N-1)
        range_y = range(1, M-1)
    elif conditions == 2:
        range_x = range(2, N-2)
        range_y = range(2, M-2)
    else:
        raise ValueError("Invalid number of conditions. Use 1 or 2.")

    
    # Define 3 different methods for solving the Poisson equation
    # 1. Point Jacobi relaxation
    # 2. SOR (Successive Over-Relaxation)
    # 3. Gauss-Seidel
    if method == 'point Jacobi relaxation':
        for i in range_x:
            for j in range_y:
                p[i, j] += beta * rp[i, j] / b_P
    # elif method == 'SOR':
    #     for i in range(1, N-1):
    #         for j in range(1, M-1):
    #             p[i, j] += beta * (rp[i, j] / b_P + 
    #                                (b_W * p[i-1, j] + b_E * p[i+1, j] +
    #                                 b_S * p[i, j-1] + b_N * p[i, j+1]))
    # elif method == 'Gauss-Seidel':
    #     for i in range(1, N-1):
    #         for j in range(1, M-1):
    #             p[i, j] = (b_W * p[i-1, j] + b_E * p[i+1, j] +
    #                        b_S * p[i, j-1] + b_N * p[i, j+1] - rp[i, j]) / b_P
    else:
        raise ValueError("Unknown method: {}".format(method))
    
    return p

# Implement updating of the vorticity function at the boundary $\omega_{1,j}, \omega_{M,j}, \omega_{i,1}, \omega_{i,N}$

## 🔹 $2^{nd}$ order approximation
$$
\omega_{1,j} = \frac{3\psi_{2,j}}{\Delta x^2} - \frac{1}{2}\omega_{2,j} 
$$
$$
\omega_{N,j} = \frac{3\psi_{N-1,j}}{\Delta x^2} - \frac{1}{2}\omega_{N-1,j} 
$$
$$
\omega_{i,1} = \frac{3\psi_{i,2}}{\Delta y^2} - \frac{1}{2}\omega_{i,2} 
$$
$$
\omega_{i,M} = \frac{3\psi_{i,M-1}}{\Delta y^2} - \frac{1}{2}\omega_{i,M-1}
$$

In [ ]:
def BCvor(vor, p, dx, dy):
    # Update vorticity at the boundaries using 2nd order approximation
    for j in range(M):
        vor[0, j] = 3.0 * p[1, j] / dx**2 - 0.5 * vor[1, j]
        vor[N-1, j] = 3.0 * p[N-2, j] / dx**2 - 0.5 * vor[N-2, j]
    
    # Update along the horizontal boundaries (i-loop)
    for i in range(1, N-1):
        vor[i, 0] = 3.0 * p[i, 1] / dy**2 - 0.5 * vor[i, 1]
        vor[i, M-1] = 3.0 * (p[i, M-2] + dy) / dy**2 - 0.5 * vor[i, M-2]
    
    return vor  

# Implement updating of stream function $\psi_{2,j}, \psi_{M-2,j}, \psi_{i,2}, \psi_{i,N-2}$ (we assume that $\psi = \frac{\partial \psi}{\partial x} = \frac{\partial \psi}{\partial y} = 0$ at the boundary)

## 🔹 Updating stream function $\psi$ at the boundary
$$
\psi_{2,j} = \frac{1}{4} \psi_{3,j}
$$
$$
\psi_{N-1,j} = \frac{1}{4} \psi_{N-2,j}
$$
$$
\psi_{i,2} = \frac{1}{4} \psi_{i,3}
$$
$$
\psi_{i,M-2} = \frac{1}{4} \psi_{i,M-2}
$$

In [ ]:
def BCp(p):
    # Update p along the vertical boundaries
    for j in range(1, M - 1):  
        p[1, j] = 0.25 * p[2, j]  
        p[N - 2, j] = 0.25 * p[N - 3, j]  

    # Update p along the horizontal boundaries
    for i in range(1, N - 1):  
        p[i, 1] = 0.25 * p[i, 2] 
        p[i, M - 2] = 0.25 * (p[i, M - 3] - 2.0 * dy)

    return p